In [172]:
from datetime import datetime
import requests
import re
from bs4 import BeautifulSoup 
import csv

In [15]:
template = 'https://www.indeed.com/jobs?q={}&l={}'

In [16]:
def get_url(position, location):
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

In [17]:
url = get_url('data analyst', 'normal il')

In [18]:
response = requests.get(url)

In [19]:
response

<Response [200]>

In [20]:
response.reason

''

In [21]:
soup = BeautifulSoup(response.text,'html.parser')

In [83]:
cards = soup.find_all('div', 'job_seen_beacon')

In [46]:
len(cards)

15

In [84]:
card = cards[0]

In [85]:
spantag = card.h2.span

In [31]:
job_title = spantag.get('title')

AttributeError: 'NoneType' object has no attribute 'a'

In [51]:
company = card.find('span', 'companyName').text.strip()

In [52]:
job_location = card.find('div', 'companyLocation').text.strip()

In [53]:
post_date = card.find('span', 'date').text

In [54]:
today = datetime.today().strftime('%Y-%m-%d')

In [144]:
try:
    job_salary = card.find('svg', {'aria-label' : 'Salary'}).text.strip()
except AttributeError:
    job_salary = ''

In [145]:
def get_record(card):
    spantag = card.h2.span
    job_title = spantag.get('title')
    company = card.find('span', 'companyName').text.strip() 
    job_location = card.find('div', 'companyLocation').text.strip()
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('svg', {'aria-label' : 'Salary'}).text.strip()
    except AttributeError:
        job_salary = ''
    
    record = (job_title, company, job_location, post_date, today, job_salary)
    
    return record 

In [146]:
records = []

for card in cards: 
    record = get_record(card)
    records.append(record)

In [148]:
records[0]

('Corporate Business Analyst - Normal, IL',
 'CIT Trucks',
 'Normal, IL 61761',
 'Posted30+ days ago',
 '2022-01-05',
 '')

In [149]:
while True:
    try:
        url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
    except AttributeError: 
        break

    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    cards = soup.find_all('div', 'job_seen_beacon')
    
    for card in cards: 
        record = get_record(card)
        records.append(record)

In [142]:
len(records)

25

In [170]:
from datetime import datetime
import requests
import re
from bs4 import BeautifulSoup 
import csv

def get_url(position, location):
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

def get_record(card):
    spantag = card.h2.span
    job_title = spantag.get('title')
    company = card.find('span', 'companyName').text.strip() 
    job_location = card.find('div', 'companyLocation').text.strip()
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('svg', {'aria-label' : 'Salary'}).text.strip()
    except AttributeError:
        job_salary = ''
    
    record = (job_title, company, job_location, post_date, today, job_salary)
    
    return record 

def main(position, location):
    records = []
    url = get_url(position, location)
    
    
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')
        cards = soup.find_all('div', 'job_seen_beacon')

        for card in cards: 
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError: 
            break
    
    with open('job_search_results.csv', 'w', newline='', encoding='utf-8') as f:
              writer = csv.writer(f)
              writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Salary'])
              writer.writerows(records)       

In [171]:
main('data analyst', 'normal il')

0